# Vectorized query 

In [1]:
from dl import authClient as ac, queryClient as qc
from dl.helpers.utils import convert
from getpass import getpass
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.cosmology import WMAP9 as cosmo
import os.path
from scipy import stats
import requests
from astroquery.simbad import Simbad
from astropy import coordinates
from astropy.coordinates import search_around_3d

There are 4715 UDGs present

In [2]:
t = pd.read_csv('DESI_UDGs_sorted_nodups_nomisses.csv')

In [3]:
# Below defines the upper and lower limits 
# limits of magnitudes mainly used to get rid of 'nan's
upper_z = 0.02001 # redshift
lower_z = 0.015
mag_upper = -19
mag_lower = -22
#diff_mean = 0.3

In [4]:
Simbad.get_votable_fields()

['main_id', 'coordinates']

In [5]:
Simbad.add_votable_fields('z_value')
Simbad.add_votable_fields('flux(g)')  
Simbad.add_votable_fields('flux(B)') # Newly added
Simbad.add_votable_fields('ra(d)')
Simbad.add_votable_fields('dec(d)')
Simbad.remove_votable_fields('coordinates')
Simbad.get_votable_fields()

['main_id', 'z_value', 'flux(g)', 'flux(B)', 'ra(d)', 'dec(d)']

In [6]:
import random 
# The first UDG that returned df table was the 4th object on the catalog (index nth = 3)
nth = 3
# Shift the coordinate 
Shift_deg = 0 #random.uniform(60, 90), 60.88433459836645
print(Shift_deg)
co = coordinates.SkyCoord(ra=t['RA_corr'][nth]+Shift_deg , dec=t['dec_corr'][nth], unit=(u.deg, u.deg),frame='icrs')

sep_all = np.array([])
sep_red, sep_blue = [], []
test = []
print('UDG Number ', nth+1)


# Creates the very first df table, df_base, which is later used to append all the df_3s to for the rest of the UDGs using pd.merge
try:
    df = Simbad.query_region(co,radius= 1.3 * u.degree,equinox=2020.0,epoch='J2000.0')
except OSError:
    print('fail')
    pass


# calculate color of UDG    
color = t['color'][nth]

# set up UDG coordinate    
udg_coord = SkyCoord(ra=t['RA_corr'][nth]+Shift_deg, dec=t['dec_corr'][nth], unit=(u.deg, u.deg),frame='icrs')

#   implement first cut on redshift and a measured g band apparent magnitude
try:
    df_1 = df[(df['Z_VALUE']<upper_z) & (df['Z_VALUE']>lower_z) & (df['FLUX_g']>0)]# or (df['FLUX_B']>0))]
    #df_1 = df[(df['FLUX_g']>0) or (df['FLUX_B']>0)]

#    use cosmo.luminosity_distance(df['Z_VALUE'] to convert apparent to absolute magnitudes
    df_1['abs_mag'] = df_1['FLUX_g']/u.mag - (5 * np.log10(cosmo.luminosity_distance(df_1['Z_VALUE'])/u.Mpc*10**5))

#   implement second cut for MW analogs    
    df_2 = df_1[(df_1['abs_mag']<mag_upper) & (df_1['abs_mag']>mag_lower)]

#   set up coordinates for all normal galaxies    
    normal_coord =  SkyCoord(df_2['RA_d'],df_2['DEC_d'],unit=(u.deg, u.deg),frame='icrs')

#   calculate projected separation for each normal galaxy to the current UDG using angular diameter distance
    df_2['separation'] = normal_coord.separation(udg_coord)/u.deg/206265*3600*cosmo.angular_diameter_distance(df_2['Z_VALUE'])/u.Mpc  
#   implement final cut on physical projected separation    
    df_3 = df_2[(df_2['separation']<1.5)]      
    df_3['UDG id'] = t['Filename_corr'][nth]       
    list_of_neighbors = df_3['separation'].tolist()
    if (len(list_of_neighbors)>0):
#   save all projected separations that satisfy, and do the same as a function of UDG color
        print(df_3)   
        sep_all = np.append(sep_all,list_of_neighbors)   
        if (color < 0.45):
            sep_blue = np.append(sep_blue,list_of_neighbors)
        elif (color < 0.9):
            sep_red = np.append(sep_red,list_of_neighbors)

#        if (color > 0.45):
#             red.append(df_3['separation'])                             
#        elif (color < 0.9):
#             blue.append(df_3['separation'])

#        print(df_3['separation'],df_3['abs_mag'])

    #print(df_3['separation'],df_3['abs_mag'])

except ValueError:
#    print('df is 0')
    pass

0
UDG Number  4
        MAIN_ID          Z_VALUE  ...     separation           UDG id      
                                  ...                                      
----------------------- --------- ... ------------------ ------------------
2MASX J00030887+3102108 0.0161380 ... 0.6343283354307241 SMDG0000453+305356
              NGC  7806 0.0161100 ... 0.6747819270447365 SMDG0000453+305356
              UGC 12864 0.0157120 ...  0.844226257203949 SMDG0000453+305356
          MCG+05-01-027 0.0165110 ...  0.898925434182886 SMDG0000453+305356
              NGC  7819 0.0166020 ... 1.1900279777221836 SMDG0000453+305356


/data0/sw/anaconda3/lib/python3.7/site-packages/astropy/table/column.py:1152: RuntimeWarning: invalid value encountered in less
  result = getattr(super(), op)(other)
/data0/sw/anaconda3/lib/python3.7/site-packages/astropy/table/column.py:1152: RuntimeWarning: invalid value encountered in greater
  result = getattr(super(), op)(other)


In [7]:
df_base = df_3.to_pandas()
df_base

,MAIN_ID,Z_VALUE,FLUX_g,FLUX_B,RA_d,DEC_d,abs_mag,separation,UDG id
0,2MASX J00030887+3102108,0.016138,14.809,15.0,0.787037,31.036402,-19.437248,0.634328,SMDG0000453+305356
1,NGC 7806,0.016110,14.307,14.4,0.375238,31.441850,-19.935431,0.674782,SMDG0000453+305356
2,UGC 12864,0.015712,14.444,14.7,359.349975,30.992164,-19.743447,0.844226,SMDG0000453+305356
3,MCG+05-01-027,0.016511,15.199,15.6,0.709023,31.485008,-19.097487,0.898925,SMDG0000453+305356
4,NGC 7819,0.016602,14.273,14.3,1.102063,31.472028,-20.035574,1.190028,SMDG0000453+305356


In [8]:
Simbad.get_votable_fields()

['main_id', 'z_value', 'flux(g)', 'flux(B)', 'ra(d)', 'dec(d)']

In [9]:
#for i in range(len(co)): # Iterates through UDGs
for i in range(4, 4714):
    Shift_deg = 0 #random.uniform(50, 90)
    co = coordinates.SkyCoord(ra=t['RA_corr'][i]+Shift_deg, dec=t['dec_corr'][i],unit=(u.deg, u.deg),frame='icrs')
    print('UDG Number ', i)
    try:
        df = Simbad.query_region(co,radius= 1.3 * u.degree,equinox=2020.0,epoch='J2000.0')
    except OSError:
        print('fail')
        pass

# calculate color of UDG    
    color = t['color'][i]

# set up UDG coordinate    
    #udg_coord = SkyCoord(ra=t['RA_corr'][i] * u.deg, dec=t['dec_corr'][i] * u.deg, frame='icrs')
    udg_coord = SkyCoord(ra=t['RA_corr'][i]+Shift_deg, dec=t['dec_corr'][i], unit=(u.deg, u.deg),frame='icrs')
    
#   implement first cut on redshift and a measured g band apparent magnitude
    try:
        df_1 = df[(df['Z_VALUE']<upper_z) & (df['Z_VALUE']>lower_z) & (df['FLUX_g']>0) & (df['FLUX_B']>0)]
        #df_1 = df[(df['Z_VALUE']<upper_z) & (df['Z_VALUE']>lower_z) & ((df['FLUX_g']>0) or (df['FLUX_B']>0))]

    #    use cosmo.luminosity_distance(df['Z_VALUE'] to convert apparent to absolute magnitudes
        #df_1['abs_mag'] = mag/u.mag - (5 * np.log10(cosmo.luminosity_distance(df_1['Z_VALUE'])/u.Mpc*10**5))

        df_1['abs_mag'] = df_1['FLUX_g']/u.mag - (5 * np.log10(cosmo.luminosity_distance(df_1['Z_VALUE'])/u.Mpc*10**5))

    #   implement second cut for MW analogs    
        df_2 = df_1[(df_1['abs_mag']<mag_upper) & (df_1['abs_mag']>mag_lower)]

    #   set up coordinates for all normal galaxies    
        normal_coord =  SkyCoord(df_2['RA_d'],df_2['DEC_d'],unit=(u.deg, u.deg),frame='icrs')
                                          
#   calculate projected separation for each normal galaxy to the current UDG using angular diameter distance
        df_2['separation'] = normal_coord.separation(udg_coord)/u.deg/206265*3600*cosmo.angular_diameter_distance(df_2['Z_VALUE'])/u.Mpc  
    #   implement final cut on physical projected separation    
        df_3 = df_2[(df_2['separation']<1.5)]      
        df_3['UDG id'] = t['Filename_corr'][i]       
        list_of_neighbors = df_3['separation'].tolist()
#   save all projected separations that satisfy, and do the same as a function of UDG color
        if (len(list_of_neighbors)>0):
            sep_all = np.append(sep_all,list_of_neighbors)   
            if (color < 0.45):
                sep_blue = np.append(sep_blue,list_of_neighbors)
            elif (color < 0.9):
                sep_red = np.append(sep_red,list_of_neighbors)
 #   converts df_3 from astropy table to panda DataFrame
#            print(df_3)
            df_3 = df_3.to_pandas()
#    merge df_3s and the exisiting table 
            result = pd.concat([df_base, df_3]) 
            df_base = result

    #        if (color > 0.45):
    #             red.append(df_3['separation'])                             
    #        elif (color < 0.9):
    #             blue.append(df_3['separation'])

            print(df_3['separation'],df_3['abs_mag'])

        #print(df_3['separation'],df_3['abs_mag'])
        
    except ValueError:
        print('df is 0')
        pass
    except TypeError:
        print('none found')


UDG Number  4
df is 0
UDG Number  5
df is 0
UDG Number  6
df is 0
UDG Number  7


/data0/sw/anaconda3/lib/python3.7/site-packages/astropy/table/column.py:1152: RuntimeWarning: invalid value encountered in less
  result = getattr(super(), op)(other)
/data0/sw/anaconda3/lib/python3.7/site-packages/astropy/table/column.py:1152: RuntimeWarning: invalid value encountered in greater
  result = getattr(super(), op)(other)
/data0/sw/anaconda3/lib/python3.7/site-packages/astropy/table/column.py:1152: RuntimeWarning: invalid value encountered in less
  result = getattr(super(), op)(other)
/data0/sw/anaconda3/lib/python3.7/site-packages/astropy/table/column.py:1152: RuntimeWarning: invalid value encountered in greater
  result = getattr(super(), op)(other)
/data0/sw/anaconda3/lib/python3.7/site-packages/astropy/table/column.py:1152: RuntimeWarning: invalid value encountered in less
  result = getattr(super(), op)(other)
/data0/sw/anaconda3/lib/python3.7/site-packages/astropy/table/column.py:1152: RuntimeWarning: invalid value encountered in greater
  result = getattr(super(), 

df is 0
UDG Number  8
df is 0
UDG Number  9
df is 0
UDG Number  10
df is 0
UDG Number  11
df is 0
UDG Number  12


/data0/sw/anaconda3/lib/python3.7/site-packages/astropy/table/column.py:1152: RuntimeWarning: invalid value encountered in less
  result = getattr(super(), op)(other)
/data0/sw/anaconda3/lib/python3.7/site-packages/astropy/table/column.py:1152: RuntimeWarning: invalid value encountered in greater
  result = getattr(super(), op)(other)
/data0/sw/anaconda3/lib/python3.7/site-packages/astropy/table/column.py:1152: RuntimeWarning: invalid value encountered in less
  result = getattr(super(), op)(other)
/data0/sw/anaconda3/lib/python3.7/site-packages/astropy/table/column.py:1152: RuntimeWarning: invalid value encountered in greater
  result = getattr(super(), op)(other)
/data0/sw/anaconda3/lib/python3.7/site-packages/astropy/table/column.py:1152: RuntimeWarning: invalid value encountered in less
  result = getattr(super(), op)(other)
/data0/sw/anaconda3/lib/python3.7/site-packages/astropy/table/column.py:1152: RuntimeWarning: invalid value encountered in greater
  result = getattr(super(), 

df is 0
UDG Number  13
df is 0
UDG Number  14
df is 0
UDG Number  15
df is 0
UDG Number  16
df is 0
UDG Number  17


...

/data0/sw/anaconda3/lib/python3.7/site-packages/astropy/table/column.py:1152: RuntimeWarning: invalid value encountered in less
  result = getattr(super(), op)(other)
/data0/sw/anaconda3/lib/python3.7/site-packages/astropy/table/column.py:1152: RuntimeWarning: invalid value encountered in greater
  result = getattr(super(), op)(other)
/data0/sw/anaconda3/lib/python3.7/site-packages/astropy/table/column.py:1152: RuntimeWarning: invalid value encountered in less
  result = getattr(super(), op)(other)
/data0/sw/anaconda3/lib/python3.7/site-packages/astropy/table/column.py:1152: RuntimeWarning: invalid value encountered in greater
  result = getattr(super(), op)(other)
/data0/sw/anaconda3/lib/python3.7/site-packages/astropy/table/column.py:1152: RuntimeWarning: invalid value encountered in less
  result = getattr(super(), op)(other)
/data0/sw/anaconda3/lib/python3.7/site-packages/astropy/table/column.py:1152: RuntimeWarning: invalid value encountered in greater
  result = getattr(super(), 

df is 0
UDG Number  4705
df is 0
UDG Number  4706
df is 0
UDG Number  4707
df is 0
UDG Number  4708
df is 0
UDG Number  4709


/data0/sw/anaconda3/lib/python3.7/site-packages/astropy/table/column.py:1152: RuntimeWarning: invalid value encountered in less
  result = getattr(super(), op)(other)
/data0/sw/anaconda3/lib/python3.7/site-packages/astropy/table/column.py:1152: RuntimeWarning: invalid value encountered in greater
  result = getattr(super(), op)(other)
/data0/sw/anaconda3/lib/python3.7/site-packages/astropy/table/column.py:1152: RuntimeWarning: invalid value encountered in less
  result = getattr(super(), op)(other)
/data0/sw/anaconda3/lib/python3.7/site-packages/astropy/table/column.py:1152: RuntimeWarning: invalid value encountered in greater
  result = getattr(super(), op)(other)
/data0/sw/anaconda3/lib/python3.7/site-packages/astropy/table/column.py:1152: RuntimeWarning: invalid value encountered in less
  result = getattr(super(), op)(other)
/data0/sw/anaconda3/lib/python3.7/site-packages/astropy/table/column.py:1152: RuntimeWarning: invalid value encountered in greater
  result = getattr(super(), 

df is 0
UDG Number  4710
df is 0
UDG Number  4711
df is 0
UDG Number  4712
df is 0
UDG Number  4713
df is 0


/data0/sw/anaconda3/lib/python3.7/site-packages/astropy/table/column.py:1152: RuntimeWarning: invalid value encountered in less
  result = getattr(super(), op)(other)
/data0/sw/anaconda3/lib/python3.7/site-packages/astropy/table/column.py:1152: RuntimeWarning: invalid value encountered in greater
  result = getattr(super(), op)(other)
/data0/sw/anaconda3/lib/python3.7/site-packages/astropy/table/column.py:1152: RuntimeWarning: invalid value encountered in less
  result = getattr(super(), op)(other)
/data0/sw/anaconda3/lib/python3.7/site-packages/astropy/table/column.py:1152: RuntimeWarning: invalid value encountered in greater
  result = getattr(super(), op)(other)


In [10]:
np.array(result['FLUX_g']), np.array(result['FLUX_B'])
diff = result['FLUX_B']-result['FLUX_g']
np.average(diff)

0.36124462